MS peak align 

In [44]:
#Loading Modules
import pandas as pd #handling of csv files and dataframes
import numpy as np
from msalign import msalign

In [45]:
# print working directory
import os
print(os.getcwd())


c:\Users\sinegraa\Downloads


In [50]:
#loading data 
data = pd.read_excel('Week2_Trimmed.xlsx')
print(data.head(10))

   Unnamed: 0 SampleName    Vial  Retention Time       Area     Height  \
0           9  2Week_40C   1:A,1          12.673   217673.0     4168.0   
1          10  2Week_40C   1:A,1           9.939    16884.0     3738.0   
2          13  2Week_40C   1:A,1           8.904  4405299.0  1029144.0   
3         135  2Week_40C  1:A,10           8.906  4743392.0  1098051.0   
4         136  2Week_40C  1:A,10           9.940    18490.0     4110.0   
5         118  2Week_40C  1:A,11           8.914  4287113.0   998123.0   
6         119  2Week_40C  1:A,11           9.949    16371.0     3642.0   
7         123  2Week_40C  1:A,12           8.918  5890075.0  1357347.0   
8         126  2Week_40C  1:A,12           9.946    22715.0     5008.0   
9          23  2Week_40C   1:A,2           9.939    24013.0     5314.0   

  Peak Type          Name  Injection  \
0   Unknown           NaN          1   
1     Found  API_Impurity          1   
2     Found           API          1   
3   Unknown           NaN

In [51]:
#split vial column into plate and coordinate columns by the : in the vial column and remove the original vial column
data[['Plate','Coordinate']] = data['Vial'].str.split(':',expand=True)
#remove comma from coordinate column entries
data['Coordinate'] = data['Coordinate'].str.replace(',','')


at this point, prompt user to ask which vial you want to align the peaks to, then call the retention time of those peaks so we can put them into the MSAlign package

In [52]:
#promt user to ask which coordinate to align the data to
print('Which coordinate would you like to align the data to?')
print('Options are: A1, B2...')
coordinate = input('Enter your choice: ')


Which coordinate would you like to align the data to?
Options are: A1, B2...


In [53]:
# Filter the data based on the selected coordinate
filtered_data = data[data['Coordinate'] == coordinate]

# Sort the filtered data by area in descending order
sorted_data = filtered_data.sort_values('Area', ascending=False)

# Get the retention times of the top four peaks
retention_times = sorted_data['Retention Time'].head(4).tolist()

# Print the retention times
retention_times

[8.9, 8.911, 9.925, 9.939]

In [55]:
#create new dataframe from data, only with retention time, sample name and area columns
#using Height as a placeholder for now
datafiltered = data[['Retention Time', 'SampleName', 'Height']]

In [ ]:
reversed_data = data.pivot(index='Retention Time', columns='SampleName', values='Area')
reversed_data.columns.name = None
reversed_data.reset_index(inplace=True)

In [ ]:
#convert to numpy array
datanp = data.to_numpy()
x = datanp[1:,0]
array = datanp[1:,1:].T

In [ ]:
# run the msalign function with the retention times
aligned_array = msalign(x,array, retention_times, weights=[100,100], only_shift=False)


ValueError: could not convert string to float: '2Week_40C'

In [ ]:
# start by storing file locally in my local directory
# once we have this working, we can replace this with asking the user to upload the files
#focus on saving the output to the same directory as the input file


I am in favor of getting rid of what's below 

In [ ]:
# This part was an attempt of having an interface to click and select files 
#It does currently not work

from tk import  Checkbutton, Button, IntVar

def get_selected_items():
    global csv_file_path
    selected_items = [item.get() for item in checkbox_vars]
    csv_file_path = []
    print("Selected items:", selected_items)
    for j,counter in enumerate(selected_items):
        if counter==True:
            csv_file_path.append(filedialog.askopenfilename(title="Select CSV File for ", filetypes=[("CSV Files", "*.csv")]))
            # Display the selected file path
            print("Selected CSV File for ",checkbox_names[j], " is: ", csv_file_path[j])
        else: csv_file_path.append("0")
    root.quit()

# Create the Tkinter root window
root = Tk()

# Create tkinter IntVar variables to store the state of the checkboxes
checkbox_vars = [IntVar() for _ in range(6)]
checkbox_names = ["Week0","Week0_humid","Week2","Week2_humid","Week5","Week5_humid"]

# Create the checkboxes
checkboxes = []
for i, var in enumerate(checkbox_vars):
    checkbox = Checkbutton(root, text=checkbox_names[i], variable=var)
    checkbox.pack()
    checkboxes.append(checkbox)

# Create a button to get the selected items
button = Button(root, text="Get Selected Items", command=get_selected_items)
button.pack()

# Run the Tkinter event loop
root.mainloop()

ImportError: cannot import name 'Checkbutton' from 'tk' (C:\Users\sinegraa\AppData\Roaming\Python\Python310\site-packages\tk\__init__.py)

In [ ]:
#Loading of xlsx-files and reformatting of dataframe - Week2 loaded as example
df = pd.read_excel("C:/Users/dejejord/OneDrive - Merck Sharp & Dohme LLC/Desktop/ExcipientCompatibility_Jupyter/Week2_Trimmed.xlsx")

#Trim down to most important columns
df1 = df[["Vial","Retention Time", "Area"]]

#Get relative retention times by division by retention time of largest area peak ==> Caveat: largest area needs to be API peak
max_area = df1["Area"].max()
max_area_row=df1[df1["Area"] == max_area]
max_Ret_value = max_area_row["Retention Time"].iloc[0]
df1["Retention Time"] = df1["Retention Time"] / max_Ret_value

#Pivot dataframe to be sorted by retention times
df1=df1.round(2)
pivot_df1 = df1.pivot_table(index="Vial", columns="Retention Time", values="Area",aggfunc='sum')
pivot_df1[np.isnan(pivot_df1)] = 0

#Merge relative retention times if difference is 0.01 and no entry recorded for one of two columns for each well
df_merged = pd.DataFrame()
col_del = []
columns = pivot_df1.columns.tolist()
c=0

while c <= len(columns)-2:
    merge_condition = False
    for index,row in pivot_df1.iterrows():
        if round((columns[c+1]-columns[c]),2)==0.01:
            if (row[columns[c+1]] != 0.0 and row[columns[c]] == 0.0) or (row[columns[c+1]] == 0.0 and row[columns[c]] != 0.0):
                merge_condition = True
                df_merged[columns[c]] =  pivot_df1[columns[c]]+pivot_df1[columns[c+1]]
                c=c+2
                break
    if merge_condition == False:
        df_merged[columns[c]] =  pivot_df1[columns[c]]
        c=c+1
if c == len(columns)-1:
    df_merged[columns[c]] =  pivot_df1[columns[c]]


#Normalize by API peak area
final_norm = df_merged.div(df_merged[df_merged.columns[df_merged.columns == 0.99]].values,axis=0)
final_norm=final_norm.round(5)

In [ ]:
df.set_index(["Vial"])["SampleName"]

In [ ]:
non_zero_counts = np.count_nonzero(df_merged, axis=0)
result = pd.Series(non_zero_counts, index=df_merged.columns)

print(result)

In [ ]:
final_norm